In [ ]:
!pip install dask-cuda

In [ ]:
# !pip install xgboost==1.0.0


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!nvidia-smi

Sat Jan 22 15:39:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !pip install dask[dataframe]

In [ ]:
!pip install optuna

     |████████████████████████████████| 308 kB 4.0 MB/s 
     |████████████████████████████████| 80 kB 13.1 MB/s 
     |████████████████████████████████| 209 kB 87.0 MB/s 
     |████████████████████████████████| 75 kB 6.4 MB/s 
     |████████████████████████████████| 49 kB 8.4 MB/s 
     |████████████████████████████████| 149 kB 93.3 MB/s 
     |████████████████████████████████| 112 kB 90.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=c4ea4a1cc7d3f34fa8b5934b774bcab9e14a95e7b16efadb303af591aa5c61ba
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.preprocessing import RobustScaler
from tqdm import tqdm

import optuna
import gc
xgb.__version__

'0.90'

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/test.csv')
columns = [col for col in train.columns.to_list() if col not in ['id','song_popularity']]
data=train[columns]
target=train['song_popularity']

In [ ]:
def objective(trial):
  train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
  # params = {
  #       'objective': trial.suggest_categorical('objective',['multi:softprob']), 
  #       'num_class': trial.suggest_categorical('num_class',[2]), 
  #       'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
  #       'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
  #       'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
  #       'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
  #       'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
  #       'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
  #       'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
  #       'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
  #       'eval_metric': trial.suggest_categorical('eval_metric',['auc_roc']),
  # }
  params = {
        "verbosity": 1,
        "objective": trial.suggest_categorical('objective',['multi:softprob']), 
        "n_estimators": 2000,
        'num_class': trial.suggest_categorical('num_class',[2]), 
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        "subsample": trial.suggest_loguniform("subsample", 0.4, 0.8),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 10, 1000),
        "seed": 42,
        
    }
  model = xgb.XGBClassifier(**params)
  model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=300,verbose=False)
  preds = model.predict(test_x)
  roc = roc_auc_score(test_y, preds)
  return roc

In [ ]:
study = optuna.create_study(direction='maximize',study_name='song-pop_trials_200')
study.optimize(objective, n_trials=200)

[I 2022-01-22 15:43:12,923] A new study created in memory with name: song-pop_trials_200
[I 2022-01-22 15:43:42,939] Trial 0 finished with value: 0.513111293182301 and parameters: {'objective': 'multi:softprob', 'num_class': 2, 'max_depth': 5, 'learning_rate': 0.028580782748713444, 'colsample_bytree': 0.3076695663907463, 'subsample': 0.6498455977651223, 'alpha': 0.03451435895048032, 'lambda': 1.9254755103258865e-06, 'min_child_weight': 15.934415116516286}. Best is trial 0 with value: 0.513111293182301.
[I 2022-01-22 15:44:04,871] Trial 1 finished with value: 0.5098090853523728 and parameters: {'objective': 'multi:softprob', 'num_class': 2, 'max_depth': 7, 'learning_rate': 0.04536840226134837, 'colsample_bytree': 0.2091495340341043, 'subsample': 0.5397578989559941, 'alpha': 0.12110384692738035, 'lambda': 4.671295615557198e-07, 'min_child_weight': 32.02382704008961}. Best is trial 0 with value: 0.513111293182301.
[I 2022-01-22 15:44:26,039] Trial 2 finished with value: 0.5123183442503796

In [ ]:
study.best_value

In [ ]:
study.best_params